In [1]:
import numpy as np

In [2]:
for i in range(1,6):
    print(i)

1
2
3
4
5


In [3]:
l =[1, 22, 43,64, 90]
for i in range(1, len(l)):
    print(l[i], l[i-1])

22 1
43 22
64 43
90 64


In [4]:
def Initializing_parameters(layer_dims):     #(no.of.input, #unit.in.layer.1, ...., #unit.in.layer.L-1)
    
    parameters = {}  #create Dictionary
    
    for i in range(1,len(layer_dims)):
        """create W and b with (l, l-1) and (l, 1) dim respectively"""
        #print(layer_dims[i], layer_dims[i-1]) #debug
        parameters['W' + str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1]) * 0.01
        parameters['b' + str(i)] = np.zeros((layer_dims[i], 1))
     
    #***
    #debug
    #for key in parameter:
        #print(key)
    #***
        
    return parameters

For Forward Propogation need following

$Z^{[l]}$ ,$A^{[l-1]}$, $W^{[l]}$, $b^{[l]}$, $g^{[l]}(Z^{[l]})$


if $g^{[l]}(Z^{[l]})$ is relu then
\begin{align}
g^{[l]}(Z^{[l]}) = max(0,Z^{[l]})
\end{align}

if $g^{[l]}(Z^{[l]})$ is sigmoid then
\begin{align}
g^{[l]}(Z^{[l]}) = \frac{1}{1+e^{-Z^{[l]}}}
\end{align}

In [5]:
def Forwar_propogation(X, parameters, Layers):
    
    #no_of_features = X.shape[0]
    #Layers = [no_of_features,7,5,3,1] #define no_of_units_in_each_layer
    
    L =len(Layers)
    
    #parameters = Initializing_parameters(Layers)
    
    A_prev = X
    
    for i in range(1, L-1):
        #print('Layer', i) #debug
        """Update dictionary parameters with Z = WA + b"""
        """Activation function for layers from 1 to L-1 is relu"""
        
        parameters['Z' + str(i)] = np.dot(parameters['W' + str(i)], A_prev) + parameters['b' + str(i)]
        O = np.zeros(parameters['Z' + str(i)].shape)
        #print(O) #debug
        parameters['A' + str(i)] = np.maximum(O, parameters['Z' + str(i)])  #relu = max(0, Z)
        A_prev = parameters['A' + str(i)]
        
        #***debug
        #print(parameters['Z' + str(i)].shape,'=' ,parameters['W' + str(i)].shape,A_prev.shape)
        #A_prev = parameters['Z' + str(i)] #debug
        #parameters['A' + str(i)] = parameters['Z' + str(i)] #debug
        #***
        
    #***debug
    #print('Layer', L-1)
    #print('=' ,parameters['W' + str(L-1)].shape)
    
    #***
    
    parameters['Z' + str(L-1)] = np.dot(parameters['W' + str(L-1)], parameters['A' + str(L-2)]) + parameters['b' + str(L-1)]
    parameters['A' + str(L-1)] = 1 / (1 + np.exp(-parameters['Z' + str(L-1)]))  #sigmoid for last layer
    
    #***debug
    #for key in parameters:
    #    print(key)
    #***

    
    return parameters

In [6]:
### Test Purpose
""
X = np.array([[1, 2],
             [3, 4],
             [5, 6]])

no_of_features = X.shape[0]
Layers = [no_of_features,7,5,3,1]
parameters = Initializing_parameters(Layers)


parameters = Forwar_propogation(X, parameters,Layers)

for key in parameters:
    print(key,parameters[str(key)].shape)
""

W1 (7, 3)
b1 (7, 1)
W2 (5, 7)
b2 (5, 1)
W3 (3, 5)
b3 (3, 1)
W4 (1, 3)
b4 (1, 1)
Z1 (7, 2)
A1 (7, 2)
Z2 (5, 2)
A2 (5, 2)
Z3 (3, 2)
A3 (3, 2)
Z4 (1, 2)
A4 (1, 2)


''

In [7]:
def Calculate_cost(AL, Y):
    
    m = Y.shape[1]
    
    cost = -(1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1-Y), np.log(1-AL).T))
    
    return cost

In [8]:
for l in reversed(range(1, 6)):
    print(l)

5
4
3
2
1


In [9]:
ex_dic = {'a': np.arange(0,3).reshape(1,3),
         'b': [[5,6,7]]}
a = ex_dic['a']
print(a,'\n',a.T)


[[0 1 2]] 
 [[0]
 [1]
 [2]]


In [10]:
def Backward_popogation(X,Y, parameters):
    """need to update W, L based on learning rate using gradient decent"""
    
    """last layer has sigmoid activation function, and rest have relu"""
    
    """dZL, dZ, dW, db, dA"""
    
    grads = {} #creating dictionary for gradient values
    
    m = Y.shape[1]
    L = 4
    
    parameters['A0'] = X
    
    #--- for last layer (L)
    AL = parameters['A' + str(L)]
    grads['dZ' + str(L)] = AL - Y
    
    dZL = grads['dZ' + str(L)]
    AL_1 = parameters['A' + str(L-1)]
    grads['dW' + str(L)] = (1/m) * np.dot(dZL, AL_1.T)
    
    grads['db' + str(L)] = (1/m) * np.sum(dZL, axis = 1, keepdims=True)
    
    W = parameters['W' + str(L)]
    grads['dA' + str(L-1)] = np.dot(W.T, dZL)
    #-----****
    
    #--- for layers 1 to L-1
    for l in reversed(range(1, L)):
        
        W1 =  parameters['W' + str(l+1)]
        dZ1 = grads['dZ' + str(l+1)]
        grads['dZ' + str(l)] = np.dot(W1.T, dZ1)
        
        
        dZ = grads['dZ' + str(l)]
        A_1 = parameters['A' + str(l-1)]
        grads['dW' + str(l)] = (1/m) * np.dot(dZ, A_1.T)
        
        grads['db' + str(l)] = (1/m) * np.sum(dZ, axis = 1, keepdims=True)
        
        W = parameters['W'+str(l)]
        grads['dA' + str(l-1)] = np.dot(W.T, dZ)
        
    #-----****
    
    return grads

In [11]:
def update_parameters(parameters, grads, Learning_rate):
    
    L = 4
    
    for i in range(1, L+1):
        parameters['W' + str(i)] = parameters['W' + str(i)] - Learning_rate * grads['dW' + str(i)]
        parameters['b' + str(i)] = parameters['b' + str(i)] - Learning_rate * grads['db' + str(i)]
    
    return parameters

In [12]:
def Neural_Network_Model(X, Y, No_of_iterations, Learning_rate):
    
    no_of_features = X.shape[0]
    Layers = [no_of_features,1000,1000,200,1] #define no_of_units_in_each_layer
 
    set_parameters = Initializing_parameters(Layers)
    
    L = 4
    
    for i in range(0, No_of_iterations):
        
        ### calculate W, B, Z, and A for all L layers
        parameters = Forwar_propogation(X, set_parameters, Layers)
        
        #.... Calculte Cost
        AL = parameters['A' + str(L)]
        cost = Calculate_cost(AL, Y)
        
        #.... Backward Propogation
        grads = Backward_popogation(X, Y, parameters)
        
        set_parameters = update_parameters(parameters, grads, Learning_rate)
        
        if i%1000 == 0:
            print("for",i,"th iteration",cost)
            #print(AL.shape)
        
    return parameters

In [50]:
def predict(X, Y, parameters, Layers):
    
    m = len(Layers)-1
    param = Forwar_propogation(X, parameters, Layers)
    
    Y_pred = []
    
    Y_pred = param['A'+str(m)]
    
    
    return Y_pred

In [14]:
###-----******YES....!!!!,,,, Its Time to Test....******###

### Arrange the Data and Preprocess it,,,

import h5py

def load_dataset():
    train_dataset = h5py.File('Downloads/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('Downloads/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

### retrive images
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = load_dataset()

### Flatten them into matrix form
train_set_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T

### Normalizing the matrices
train_set_x = train_set_flatten/255
test_set_x = test_set_flatten/255

In [15]:
print('trian set shape(No_of_features, No_of_examples)',train_set_x.shape)
print('test set shape(No_of_features, No_of_examples)', test_set_x.shape)

trian set shape(No_of_features, No_of_examples) (12288, 209)
test set shape(No_of_features, No_of_examples) (12288, 50)


In [16]:
parameters = Neural_Network_Model(train_set_x, train_set_y_orig, 10000, 0.005)

for 0 th iteration [[0.69372398]]
for 1000 th iteration [[0.64405779]]
for 2000 th iteration [[0.63003872]]
for 3000 th iteration [[0.57049065]]
for 4000 th iteration [[0.34170866]]
for 5000 th iteration [[0.01615204]]
for 6000 th iteration [[0.00390624]]
for 7000 th iteration [[0.001967]]
for 8000 th iteration [[0.00125093]]
for 9000 th iteration [[0.00089406]]


In [17]:
for key in parameters:
    print(key)

W1
b1
W2
b2
W3
b3
W4
b4
Z1
A1
Z2
A2
Z3
A3
Z4
A4
A0


In [51]:
Y_pred = predict(train_set_x, train_set_y_orig, parameters, Layers = [train_set_x.shape[0],7,5,3,1])

In [52]:
Y_pred = Y_pred > 0.5

In [53]:
np.mean(Y_pred == train_set_y_orig)

1.0